In [2]:
import os 
import json
from tqdm import tqdm 
import re

In [4]:
test_out_times_path = "/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases3/train_out_times.json"
with open(test_out_times_path, 'r') as f:
    datasets = json.load(f)

print(type(datasets))
print(len(datasets))

<class 'dict'>
2057


In [5]:
def make_pair_cross(sorted_submissions):
    length = len(sorted_submissions)
    pairs = []
    for i in range(length):
        for j in range(i+1, length):
            slow, fast = i, j
            pair = (sorted_submissions[slow], sorted_submissions[fast])
            pairs.append(pair)
            
    return pairs

In [6]:
def relative_improvement(slow:float, fast:float):
    return round((slow - fast) / slow, 4)

In [43]:
# by user
items = []
for problem_id, value in tqdm(datasets.items()):
    # value is a dict
    for user_id, submissions in value.items():
        # submissions is a dict: {'s494615146': 0.001053309444, 's487731918': 0.0010449860355}
        sorted_submission_id = sorted(submissions, key=submissions.get, reverse=True) # llist
        if len(sorted_submission_id) > 1:
            pairs = make_pair_cross(sorted_submission_id)
            for each_pair in pairs:
                slow_submission_id, fast_submission_id = each_pair
                slow_time = submissions[slow_submission_id]
                fast_time = submissions[fast_submission_id]
                improvement = relative_improvement(slow_time, fast_time)
                item = {
                    "problem_id": problem_id,
                    "user_id": user_id,
                    "slow_submission_id": slow_submission_id,
                    "fast_submission_id": fast_submission_id,
                    "slow_time": slow_time,
                    "fast_time": fast_time,
                    "improvement": improvement
                }
                items.append(item)

print(f"pairs length = {len(items)}")
with open("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases3/train_out_pairs.json", 'w') as g:
    json.dump(items, g, indent=4)

100%|██████████| 2057/2057 [00:00<00:00, 5399.51it/s]


pairs length = 233554


In [7]:
datas = []
with open("/home/tongye/code_generation/pie-perf/data/cpp_original/train.jsonl", 'r') as pp:
    for line in pp:
        item = json.loads(line)
        datas.append(item)

original_pairs = []
for item in datas:
    problem_id = item["problem_id"]
    user_id = item["user_id"]
    slow_submission_id = item["submission_id_v0"]
    fast_submission_id = item["submission_id_v1"]
    pair = f"{problem_id}_{user_id}_{slow_submission_id}_{fast_submission_id}"
    original_pairs.append(pair)

print(f"length of original pairs = {len(original_pairs)}")


length of original pairs = 88136


In [8]:
with open("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases3/train_out_pairs.json", 'r') as g:
    pairs = json.load(g)

print(len(pairs))
count = 0
pairs_in_improvement10 = []
for pair in tqdm(pairs):
    problem_id = pair["problem_id"]
    user_id = pair["user_id"]
    slow_submission_id = pair["slow_submission_id"]
    fast_submission_id = pair["fast_submission_id"]
    improvement = pair["improvement"]
    identifier = f"{problem_id}_{user_id}_{slow_submission_id}_{fast_submission_id}"
    if improvement > 0.1:
        count += 1
        pairs_in_improvement10.append(pair)
print(count)

with open("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases3/train_out_pairs_improvment10.json", 'w') as p:
    json.dump(pairs_in_improvement10, p, indent=4)

233554


100%|██████████| 233554/233554 [00:00<00:00, 1439793.28it/s]

56086


In [56]:
with open("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases3/train_out_pairs.json", 'r') as g:
    pairs = json.load(g)

count = 0
for pair in tqdm(pairs):
    problem_id = pair["problem_id"]
    user_id = pair["user_id"]
    slow_submission_id = pair["slow_submission_id"]
    fast_submission_id = pair["fast_submission_id"]
    improvement = pair["improvement"]
    if improvement > 0.1:
        
        count += 1

print(count)



100%|██████████| 233554/233554 [00:00<00:00, 1918031.99it/s]

56086


In [11]:
with open("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases3/train_out_pairs_improvement10.json", 'r') as p:
    datas = json.load(p)

print(len(datas))
speedups = []
items_new = []
for item in tqdm(datas):
    problem_id = item["problem_id"]
    user_id = item["user_id"]
    slow_submission_id = item["slow_submission_id"]
    fast_submission_id = item["fast_submission_id"]
    slow_time = item["slow_time"]
    fast_time = item["fast_time"]
    improvement = item["improvement"]

    slow_cpp_file_path = os.path.join("/data3/tydata3/code_optimization/cpp/cpp_code/train", f"{problem_id}_{slow_submission_id}_{user_id}.cpp")
    with open(slow_cpp_file_path, 'r') as f_slow:
        slow_cpp_content = f_slow.read()
    slow_code = slow_cpp_content

    fast_cpp_file_path = os.path.join("/data3/tydata3/code_optimization/cpp/cpp_code/train", f"{problem_id}_{fast_submission_id}_{user_id}.cpp")
    with open(fast_cpp_file_path, 'r') as f_fast:
        fast_cpp_content = f_fast.read()
    fast_code = fast_cpp_content

    item["slow_code"] = slow_code
    item["fast_code"] = fast_code

    items_new.append(item)

with open("/data3/tydata3/code_optimization/cpp/dataset/by_user/train_out_pair_improvement10.json", 'w') as ff:
    json.dump(items_new, ff, indent=4)


56086


100%|██████████| 56086/56086 [01:28<00:00, 631.27it/s] 


In [3]:
with open("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases3/train_out_pairs.json", 'r') as p:
    datas = json.load(p)

print(len(datas))
speedups = []
for item in datas:
    if item["improvement"] > 0.1:
        speedup = item["slow_time"] / item["fast_time"]
        speedups.append(speedup)

print(sum(speedups)/len(speedups))
print(len(speedups))

233554
4.037934317541367
56086
